In [1]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import tensorflow as tf
import numpy as np
slim=tf.contrib.slim
if __name__ == "__main__":
    sys.path.append("../../../")
from notebooks_src.losses.utils import abs_smooth as smooth_L1

from notebooks_src.configs import configs
from notebooks_src.postprocessing.utils import get_int_tensor_shape, sort_some_lists_of_tensors
from notebooks_src.box_processing.match import match_boxes
from notebooks_src.box_processing.encode import encode
from notebooks_src.box_processing.unpack import unpack_net_output, split_box_class
#from utils import ssd_losses

importing Jupyter notebook from ../../../notebooks_src/losses/utils.ipynb
importing Jupyter notebook from ../../../notebooks_src/postprocessing/utils.ipynb
importing Jupyter notebook from ../../../notebooks_src/box_processing/match.ipynb
importing Jupyter notebook from ../../../notebooks_src/box_processing/tf_box_util.ipynb
importing Jupyter notebook from ../../../notebooks_src/box_processing/make_anchors_orig.ipynb
importing Jupyter notebook from ../../../notebooks_src/box_processing/encode.ipynb
importing Jupyter notebook from ../../../notebooks_src/box_processing/unpack.ipynb


In [2]:
def compute_loss(y_true, y_preds):
    '''y_true: the boxes Nx15x5 tensor
       y_preds: a list of 7?  tensors of Nxfy x fx x k where k = 4*number of anchors + number_of_anchors*num_classes,
           N is number of examples'''
    
    loss_inputs = get_loss_inputs(y_true, y_preds)
    cls_losses = {fmap_size: calc_cls_loss(fmap_size, loss_inputs) for fmap_size in loss_inputs["fmap_sizes"]}
    loc_losses = {fmap_size: calc_loc_loss(fmap_size, loss_inputs) for fmap_size in loss_inputs["fmap_sizes"]}
    
    num_matches_dict = {fmap_size: loss_inputs["match_masks"][fmap_size]["num_matches"] 
                   for fmap_size in loss_inputs["fmap_sizes"]}
    
    summary_for_all_fmaps(cls_losses, prefix="cls_loss")
    summary_for_all_fmaps(loc_losses, prefix="loc_loss")
    summary_for_all_fmaps(num_matches_dict, prefix="num_matches")
    
    
    loss = calc_loss(cls_losses,loc_losses, num_matches_dict)
    return loss
        

In [3]:
def summary_for_all_fmaps(scalar_dict, prefix=""):
    with tf.name_scope(prefix):
        for (fy,fx),v in scalar_dict.iteritems():
            tf.summary.scalar("%s_fmap_Size_%i_%i_" %(prefix,fy,fx), v)

In [4]:
def get_loss_inputs(y_true, y_preds):
    bboxes, labels = split_box_class(y_true)
    encoded_gt = encode(bboxes, labels)
    match_masks = match_boxes(bboxes)
    detections = unpack_net_output(y_preds)
    keys = detections["logits"].keys()
    return dict(detections=detections, encoded_gt=encoded_gt, match_masks=match_masks, fmap_sizes=keys)
    

In [5]:
def calc_loss(cls_losses, loc_losses, num_matches_dict):
    losses = []
    for fmap_size in cls_losses.keys():
        loc_loss = loc_losses[fmap_size]
        cls_loss = cls_losses[fmap_size]
        num_matches = num_matches_dict[fmap_size]
        match_coeff = get_match_coeff(num_matches=num_matches)
        losses.append((loc_loss + cls_loss) * match_coeff)
    total_loss = tf.add_n(losses) * (1./ configs["batch_size"])
    return total_loss

In [6]:
def calc_loc_loss(fmap_size, loss_input_dict):
    localizations, encoded_boxes, x_mask = get_loc_loss_terms(fmap_size, loss_input_dict) 
    loc_loss = smooth_L1(encoded_boxes - localizations)
    loc_loss = tf.boolean_mask(loc_loss, x_mask)
    loc_loss = tf.reduce_sum(loc_loss)
    return loc_loss



def calc_cls_loss(fmap_size, loss_input_dict ):
    clt = get_cls_loss_terms(fmap_size, loss_input_dict)
    pos_xent = get_positive_xent_loss(clt)
    neg_xent = get_negative_xent_loss(clt)
    return pos_xent + neg_xent

def get_match_coeff(num_matches):
    match_coeff = tf.where(tf.equal(num_matches, 0), 0., tf.div(1., tf.cast(num_matches, dtype=tf.float32)))
    return match_coeff


def get_loc_loss_terms(fmap_size, loss_input_dict):
    localizations = loss_input_dict["detections"]["localizations"][fmap_size]
    localizations = tf.expand_dims(localizations,axis=-1)
    
    num_coords_in_a_box = 4
    x_mask = loss_input_dict["match_masks"][fmap_size]["x_mask"]
    x_mask = tf.stack(num_coords_in_a_box*[x_mask], axis=-2)
    
    encoded_boxes = loss_input_dict["encoded_gt"][fmap_size]["boxes"]
    
    return localizations, encoded_boxes, x_mask
    

    
def get_negative_xent_loss(clt):
    encoded_labels = clt["encoded_labels"]
    num_negs = tf.cast(configs["negative_ratio"] * clt["num_matches"], dtype=tf.int32)
    num_negs = tf.where(tf.equal(num_negs,0), configs["default_negatives"], num_negs)

    hard_neg_mask = make_hard_neg_mask(clt["predictions"],
                                       clt["fp_mask"],
                                       num_negs)
    
    neg_encoded_labels = tf.where(hard_neg_mask, configs["num_classes"]*tf.ones_like(encoded_labels), encoded_labels)
    
    neg_xent_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=clt["logits"], 
                                                                   labels=neg_encoded_labels)
    neg_xent_loss = tf.boolean_mask(neg_xent_loss, hard_neg_mask)
    neg_xent = tf.reduce_sum(neg_xent_loss)
    
    return neg_xent
    

def get_positive_xent_loss(clt):
    logits, labels, x_mask = clt["logits"],clt["encoded_labels"],clt["x_mask"]
    xent_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, 
                                                               labels=labels)
    xent_loss = tf.boolean_mask(xent_loss, x_mask)
    pos_xent = tf.reduce_sum(xent_loss)
    return pos_xent
    
def get_cls_loss_terms(fmap_size, loss_input_dict):
    logits = loss_input_dict["detections"]["logits"][fmap_size]
    logits = tf.stack(configs["num_max_boxes"]*[logits], axis=-2)
    
    predictions = loss_input_dict["detections"]["predictions"][fmap_size]
    encoded_labels = loss_input_dict["encoded_gt"][fmap_size]["labels"]
    mask_dict = loss_input_dict["match_masks"][fmap_size]
    x_mask, tp_mask, actual_gt_box_mask, num_matches = [mask_dict[k] for k in ["x_mask", 
                                                                               "tp_mask", 
                                                                               "actual_gt_box_mask", 
                                                                               "num_matches"]]
    fp_mask = make_fp_mask(actual_gt_box_mask, tp_mask)
    return dict(logits=logits, encoded_labels=encoded_labels, 
                x_mask=x_mask, fp_mask=fp_mask, num_matches=num_matches, predictions=predictions)
    

In [7]:
def make_hard_neg_mask(pred, fp_mask, num_negs):
    pred = tf.stack(configs["num_max_boxes"]*[pred], axis=-2)
    float_fp_mask = tf.cast(fp_mask, dtype=pred.dtype)
    float_fp_mask = tf.expand_dims(float_fp_mask, axis=-1)
    fp_pred = pred * float_fp_mask
    last_axis = len(fp_pred.get_shape()) -1
    
    # bg class is the last element of logits
    _, bg_fp_pred = tf.split(fp_pred, num_or_size_splits=[configs["num_classes"], 1], axis=last_axis)
    flat_bg_fp_pred = tf.reshape(bg_fp_pred, [-1])
    
    num_total_negs = flat_bg_fp_pred.get_shape()[0]

    k = tf.where(num_negs > num_total_negs, num_total_negs, num_negs)
    bg_confs, inds = tf.nn.top_k(flat_bg_fp_pred,k=k)
    min_bg_conf = bg_confs[-1]
    hard_neg_mask = tf.greater_equal(bg_fp_pred, min_bg_conf)
    hard_neg_mask = tf.squeeze(hard_neg_mask, axis=-1)

    return hard_neg_mask
    
    
    

In [8]:
def make_fp_mask(actual_gt_box_mask, tp_mask):
    #is an actual box, but does not have an overlap > 0.5 with gt
    fp_mask = tf.logical_and(actual_gt_box_mask, tf.logical_not(tp_mask))
    fp_mask = tf.transpose(fp_mask, perm=(3,0,1,2,4))
    #fp_mask = tf.Print(data=[tf.reduce_sum(tf.cast(fp_mask, dtype=tf.int32))], input_=fp_mask, message="fpmask num nonzeros: ")
    return fp_mask

In [10]:
if __name__ == "__main__":
    import h5py
    with tf.Session() as sess:
        #from notebooks_src.load_data.get_generator import get_generator

        #gen=get_generator("tr", batch_size=2)
        y_true = tf.placeholder(tf.float32,shape=(5,15,5))
        box = h5py.File(configs["tr_data_file"])["boxes"][323:328]
        shapes = [(5, 6, 9, 54),
                  (5, 3, 5, 36),
                    (5, 96, 144, 36),
                    (5, 24, 36, 54),
                    (5, 12, 18, 54),
                    (5, 48, 72, 54),
                    (5, 1, 1, 36)]

        box = box.astype("float32")
        y_preds = [tf.random_normal(mean=0.0,stddev=3.,shape=shape) for shape in shapes]
        da_loss = compute_loss(y_true, y_preds)
        
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter("/home/evan/hur-detect/src/debug_logs/try1", sess.graph)
        
        loss_, summary = sess.run([da_loss,merged], feed_dict={y_true:box})
        print loss_
        writer.add_summary(summary,0)
        writer.close()
        

float32
76.1715


In [ ]:
with tf.Session() as sess:
    
    with tf.name_scope("test"):
        a = tf.constant(0)
        tf.summary.scalar("a", a)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("/home/evan/hur-detect/src/debug_logs/try2")
    summary = sess.run(merged)
    writer.add_summary(summary,0)
    

In [10]:
tf.image.draw_bounding_boxes?